# Connection diagram

This is a poster for the 22nd Workshop on Stochastic Geometry, Stereology and Image Analysis.

It's about the two-point connectivity problem for points on the boundary of a region in the Boolean model.

The picture will be of an ellipse, with two marked points on the boundary, and a path connecting them highlighted (when they're connected by a path).

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
plt.rcParams['figure.figsize'] = [10,5]
import networkx
# from PIL import Image, ImageDraw
from matplotlib.patches import Ellipse, Circle
from tqdm import trange
from IPython.display import clear_output

In [18]:
# All the functions live in here.

def sample_in_ellipse(npts, l1=1, l2=1):
    """
    Samples npts iid points uniformly at random in the ellipse with axis lengths 2*l1 and 2*l2.
    Speed's not a big issue, so we just use rejection sampling by taking points in [-l1,l1]x[-l2,l2]
    """
    pts = np.random.uniform(size=(npts,2)) # Uniform in [0,1]^2
    pts *= np.array([2*l1,2*l2])
    pts -= np.array([l1,l2])
    for i, p in enumerate(pts):
        while ( (p[0]/l1)**2 + (p[1]/l2)**2 > 1 ):
            p = np.random.uniform(size=(2)) * np.array([2*l1,2*l2]) - np.array([l1,l2])
        pts[i] = p
    return pts

def get_rgg(points_tree, r):
    """
    Returns the random geometric graph (as a networkx.Graph object)
    made from points with two points connected when their distance is less than 2r
    (i.e. the RGG version of the Boolean model with this point process).
    The vertices are 0, 1, ..., points.shape[0]-1

    Again, speed is not a major issue, so we use the crude O(n^2) method of computing adjacency.
    Actually it's a bit painfully slow even with a small number of vertices,
    so I may use something like a KDTree to speed things up.
    """
    G = networkx.Graph()
    G.add_nodes_from(range(points_tree.n))
    for i,p in enumerate(points_tree.data):
        neighbours = points_tree.query_ball_point(p,2*r)
        for j in neighbours:
            G.add_edge(i,j)
    return G

def draw_nopath(points,r,l1,l2,thetax,thetay,rgg):
    fig, ax = plt.subplots()
    region = Ellipse((0,0),2*l1,2*l2,fill=False,linewidth=2)
    ax.add_patch(region)
    ax.set_xlim(-l1-0.5,l1+0.5)
    ax.set_ylim(-l2-0.5,l2+0.5)
    plt.gca().set_aspect(1.0)
    plt.axis('off')
    circlestyle = {'linewidth':None}
    cpt1 = networkx.node_connected_component(rgg,len(points)-2)
    cpt2 = networkx.node_connected_component(rgg,len(points)-1)
    for i in cpt1:
        circle = Circle(points[i],r,color='red',**circlestyle)
        ax.add_patch(circle)
    for i in cpt2:
        circle = Circle(points[i],r,color='green',**circlestyle)
        ax.add_patch(circle)
    for i in range(len(points)):
        if not (i in cpt1 or i in cpt2):
            circle = Circle(points[i],r,color='blue',**circlestyle)
            ax.add_patch(circle)
    # for i in range(len(points)-2):
    #     circle = Circle(points[i],r,color='blue')
    #     ax.add_patch(circle)
    # for i in range(len(points)-2,len(points)):
    #     circle = Circle(points[i],r,color='red')
    #     ax.add_patch(circle)
    return

def draw_path(points,r,l1,l2,thetax,thetay,path):
    fig, ax = plt.subplots()
    region = Ellipse((0,0),2*l1,2*l2,fill=False,linewidth=2)
    ax.add_patch(region)
    ax.set_xlim(-l1-0.5,l1+0.5)
    ax.set_ylim(-l2-0.5,l2+0.5)
    plt.gca().set_aspect(1.0)
    plt.axis('off')
    for i in range(len(points)):
        if not i in path:
            circle = Circle(points[i],r,color='blue')
            ax.add_patch(circle)
    for i in path:
        circle = Circle(points[i],r,color='red')
        ax.add_patch(circle)
    return

## Finding (and drawing) the shortest path from $x$ to $y$

Now we can easily (if a bit slowly) create a random geometric graph, we'd like to find the shortest path from $x$ to $y$.

In [19]:
# Most of the parameters for the drawing
l1 = 2.5
l2 = 1
thetax = -0.75*np.pi
thetay =  0.25*np.pi
n = 10000
npts = np.random.poisson(lam=n)
print(f'With intensity {n} we have {npts} points.')

With intensity 10000 we have 9931 points.


In [20]:
points = sample_in_ellipse(npts,l1,l2)
xy = np.array([[l1*np.cos(thetax),l2*np.sin(thetax)], [l1*np.cos(thetay),l2*np.sin(thetay)]])
points = np.append(points,xy,axis=0)
xindex = npts
yindex = npts+1
points_tree = KDTree(points)

In [21]:
# r is on its own because we might want to change it more often
# The critical intensity is around 1.44
intensity = 3.460161
r = np.sqrt(intensity / n)
print(f'For n r^2 = {intensity} we have r = {r:.4f}')

For n r^2 = 3.460161 we have r = 0.0186


In [22]:
# print(f'n pi r^2 = {npts*r*r*np.log(2):.3f}')
rgg = get_rgg(points_tree,r)
# rgg.nodes[npts]['label'] = 'x'
# rgg.nodes[npts+1]['label'] = 'y'

In [23]:
try:
    path = networkx.shortest_path(rgg,xindex,yindex)
    print(path)
    is_path = True
except Exception as e:
    print(f'No path from x to y at radius {r:.4f}')
    is_path = False

[9931, 8280, 7277, 7780, 1463, 2214, 1617, 1990, 6131, 3687, 1068, 1199, 9417, 6723, 1283, 4781, 2052, 115, 2629, 9709, 4467, 1313, 1291, 4930, 8539, 5994, 3190, 4801, 647, 4019, 7870, 678, 14, 3968, 9681, 6033, 5397, 2078, 865, 6183, 619, 6429, 6411, 660, 6343, 4815, 4690, 6119, 4918, 7904, 4462, 139, 6514, 1644, 5307, 4395, 6837, 8055, 1039, 5453, 2790, 8071, 6969, 5343, 2977, 7404, 4347, 982, 8757, 3507, 632, 491, 1288, 995, 9733, 3028, 399, 242, 8083, 6903, 4795, 9619, 9563, 290, 5800, 7025, 1771, 2163, 6898, 1816, 2311, 324, 2269, 2143, 8291, 741, 9600, 3293, 6598, 3594, 8631, 3346, 3844, 9498, 1387, 2216, 4469, 8655, 236, 6586, 4150, 9361, 2730, 2679, 2344, 470, 5610, 7202, 624, 3501, 254, 261, 5188, 3521, 1221, 6358, 7691, 1299, 1026, 6680, 8175, 6466, 7152, 1685, 1209, 7099, 1395, 1708, 8048, 8520, 4448, 3000, 7207, 467, 7846, 6054, 7212, 4999, 8178, 5228, 1450, 9538, 6582, 3217, 3751, 2444, 5931, 1187, 1396, 1919, 2926, 9932]


In [24]:
# if is_path:
#     draw_path(points,r,l1,l2,thetax,thetay,path)
# else:
#     draw_nopath(points,r,l1,l2,thetax,thetay,rgg)

### Largest connected component

I'd like to colour the largest component separately (all the smaller components can be in the same hue but paler).

Also, I'd like to know roughly when $\theta(\lambda) = 1/2$, so would like to be able to tell when the size of the largest component is roughly $n/2$.

In [10]:
largest_comp = max(networkx.connected_components(rgg),key=len)
print(f'The largest component has {len(largest_comp)} vertices, roughly {100*len(largest_comp)/npts:.1f} percent of the total vertices.')

The largest component has 4544 vertices, roughly 46.4 percent of the total vertices.


In [11]:
def surveyed():
    """
    Returns a random surveyee.
    """
    options = ['tests', 'customers', 'members of the public', 'Elvis fans',
               'happy customers', 'members of the jury',
               'experts', 'expert taste-testers']
    return np.random.choice(options)

In [28]:
## Quick experiment to find when the largest component contains roughly half the discs
## Uncomment and run this bit to reset the experiment
target = 0.75
lam = 0.38 # Initial guess
trial_n = 1000
n_trials = 999
attempt = 1
step_length = 0.01

In [29]:
# Run this repeatedly to get better and better estimates
while True:
    total = 0
    votes = 0
    # for i in range(n_trials):
    for i in trange(n_trials, leave=False):
        trial_npts = np.random.poisson(trial_n)
        trial_points = sample_in_ellipse(trial_npts,l1,l2)
        trial_tree = KDTree(trial_points)
        mu = trial_n/(np.pi*l1*l2)
        trial_r = np.sqrt(lam / mu)
        trial_rgg = get_rgg(trial_tree,trial_r)
        largest_comp = max(networkx.connected_components(trial_rgg),key=len)
        votes += (len(largest_comp)/trial_npts > target)
        total += len(largest_comp)/trial_npts
    avg_theta = total / n_trials
    clear_output(wait=True)
    # if votes > 0.5*n_trials: # If we want to use voting instead of the mean size
    if avg_theta > target: # We use this mean, so the majority of votes may disagree
        lam -= step_length/attempt
        print(f'lambda = {lam:.10f} is too large, say {str(votes)} out of {str(n_trials)} {surveyed()}.')
        print(f'On average the largest component contained {100*avg_theta:.2f} percent of vertices.')
    else:
        lam += step_length/attempt
        print(f'lambda = {lam:.10f} is too small, say {str(n_trials-votes)} out of {str(n_trials)} {surveyed()}.')
        print(f'On average the largest component contained {100*avg_theta:.2f} percent of vertices.')
    print(f'On attempt {attempt}, step size {step_length/attempt:.2e}')
    attempt += 1
    if attempt > 100:
        n_trials = 99

lambda = 0.4073617164 is too small, say 425 out of 999 Elvis fans.
On average the largest component contained 74.29 percent of vertices.
On attempt 80, step size 0.0001250000


KeyboardInterrupt: 

Based on experiments so far, the halfway point happens around $\lambda \approx$ `0.36185`.

We might want to target different values for $\theta(\lambda)$ because $\theta(\lambda) = 1/2$ tends to give quite small clusters (in terms of diameter) in a thin ellipse.
Here's some values calculated with $n=1000$:
| Target $\theta(\lambda)$ | Estimated $\lambda$ | Number of attempts |
| :---: | :---: | :---: |
| $0.5$  | `0.3618508819` | $60$ |
| $0.75$ | `0.4073617164` | $80$ |
| $0.9$  | `` | $c$ |